In [1]:
!python --version


Python 3.8.5


In [51]:
import pickle

In [21]:
import pandas as pd

In [28]:
import seaborn as sns
import matplotlib as plt

In [39]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [44]:
def read_dataframe(filename):
    
    data = pd.read_parquet(filename)
    
    data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data.duration = data.duration.apply(lambda td:td.total_seconds()/60)
    
    #data = data[data.trip_type == 2]
    
    data = data[(data.duration >= 1) & (data.duration <= 60)]
    
    categorical = ['PULocationID','DOLocationID']
    
    data[categorical] =data[categorical].astype(str)
    
    return data
    

In [45]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [46]:
len(df_train),len(df_val)

(73908, 61921)

In [47]:

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [48]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [49]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [50]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479617987479523

In [52]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)